In [137]:
import os 
import pandas as pd
from pathlib import Path
import sys
import numpy 
from typing import List, Dict, Any, Optional
from charset_normalizer import from_path 

MAIN_PATH = Path().resolve().parent
sys.path.append(str(MAIN_PATH))

from src.utilities import load_config, load_validations, load_fields, load_data, init_log, format_file_size

import src.analisys 
from src.analisys import check_file_info

from src.utilities.logger import log_event

# Carrega as configurações e inicia o log 
df_config = pd.DataFrame([{}])
load_config(df_config)

# inicia o log com o path parametrizado
init_log(df_config.loc[0, "log_path"])

# Carrega a lista de validações
df_validations = pd.DataFrame([{}])
load_validations(df_validations,df_config.loc[0, "eda_config_path"] )

#print(df_validations["apply"].head()) 

# Carrega a lista de campos a validar
df_fields = pd.DataFrame([{}])
load_fields(df_fields, df_config.loc[0, "eda_config_path"])


,file,table,field,type,subtype,null,pk,fk,table_fk,format,Range,values,null_limit,active
0,ses_seguros.csv,ses_seguros,coenti,number,integer,no,yes,yes,ses_cias,NaN,NaN,NaN,NaN,yes
1,ses_cias.csv,ses_cias,coenti,number,integer,no,yes,no,NaN,NaN,NaN,NaN,NaN,yes
2,ses_ramos.csv,ses_ramos,coramo,number,integer,no,yes,no,NaN,NaN,NaN,NaN,NaN,yes
3,ses_ramos.csv,ses_ramos,banana,number,integer,no,yes,no,NaN,NaN,NaN,NaN,NaN,yes


In [138]:
result: List[Dict[str, Any]] = []
# rotina pra salvar os resutlados das analises 
def save_result(
    file: str, field: str, category: str, test: str, evidence: Any, detail: Optional[str] = None, status: str = "PASS") -> result:    
    registry: result = { "file": file, "Field": field, "category": category, "test": test, "evidence": evidence, "detail": detail, "status": status,}
    return registry

In [ ]:
# Loop nos campos
df_data = pd.DataFrame([{}])
FAIL_TABLES = set()   # Controla falhas estruturais (Passo 1)
loaded_file = ""
separator = df_config.loc[0, "separator"]
encode = df_config.loc[0, "encode"]
new_file = True 
RESULTS = [] 

# Busca checagens de nivel de arquivo 
df_file_checks = df_validations[(df_validations["apply"] == "file") & (df_validations["active"] == "yes")]

for index, row in df_fields.iterrows():

    file_name = row["file"]
    table_name = row["table"]
    file_path = df_config.loc[0, "data_path"] + file_name

    # Verifica se o arquivo está carregado ou já falhou
    if (row["file"] ==  loaded_file) or (file_name in FAIL_TABLES):
        continue
    else:         
        try: 
            # Carrega o arquivo de dados
            print("CARREGOU!" )
            df_data

            try:
                try:
                    if len(encode) == 0: 
                        detectado = from_path(str(file_path)).best() 
                        encoding_detectado = detectado.encoding 
                    else: 
                        encoding_detectado = encode
                except Exception:
                    encoding_detectado = 'utf-8' 
                df_data = pd.read_csv(
                    file_path,
                    encoding=encoding_detectado,
                    sep=separator,
                    engine='python' 
                )
            except Exception as e:
                raise ValueError(f"Falha no carregamento do arquivo !")

            loaded_file = file_name
            new_file = True  
        except UnicodeDecodeError:
            print(f"❌ Erro de Codificação: O arquivo pode não ser '{encoding}'. Tente outro encoding.")
        
        except pd.errors.ParserError as e:
            # Este erro geralmente indica problemas de estrutura (delimitadores incorretos, linhas mal formadas)
            print(f"❌ Erro de Parsing (Estrutura CSV incorreta): {e}")
        
        except Exception as e:
            # Captura outros erros, como FileNotFoundError
            print(f"❌ Erro inesperado ao ler o arquivo: {e}") 

    if new_file: 
        print("-" * 30 )
        print(table_name)
        print(file_name)
        print("-" * 30 )
        # Corrige o tipo das colunas "int" exibidas como "float" 
        for col in df_data.select_dtypes(include=["float"]).columns:
            if (df_data[col].dropna() % 1 == 0).all():
                df_data[col] = df_data[col].astype("Int64")        

        # Coleta estatisticas no nivel do arquivo 
        file_size = format_file_size(os.path.getsize(file_path) )
        line_count = len(df_data)
        col_count = len(df_data.columns) 
        evidence_str = "Tamanho: " + file_size + " Linhas/Colunas: " + str(line_count) + "/" + str(col_count)
        RESULTS.append(save_result(row["file"],  "Todos","structure","file info",evidence_str,"", "pass")) 

        # Testa colunas faltantes
        df_expected_columns = (df_fields[df_fields['table'] == table_name]['field'].unique()) 
        data_column_names = df_data.columns.to_list()
        df_data_columns = pd.DataFrame({"column": data_column_names})
        expected_set = set([c.strip().lower() for c in df_expected_columns])
        data_set = set(df_data_columns['column'].str.strip().str.lower().tolist())
        missing_colunms = expected_set.difference(data_set)
        if missing_colunms:
            str_missing_columns = ", ".join(sorted(list(missing_colunms)))
        else:
            str_missing_columns = "nenhuma"
        
        # Testa colunas com nome duplicado
        nomes_colunas = [col.strip().lower() for col in df_data.columns.tolist()]
        nomes_series = pd.Series(nomes_colunas)
        nomes_sem_sufixo = nomes_series.str.replace(r'\.\d+$', '', regex=True)
        contagem_nomes = nomes_sem_sufixo.value_counts()
        nomes_duplicados_series = contagem_nomes[contagem_nomes > 1]
        lista_nomes_duplicados = nomes_duplicados_series.index.tolist()
        colunas_duplicadas = df_data_columns['column'].value_counts()
        if lista_nomes_duplicados:
            # Junta os nomes duplicados com ", "
            resultado_string = ", ".join(lista_nomes_duplicados)
        else:
            resultado_string = "nenhum"
        if resultado_string == "nenhum" and str_missing_columns == "nenhuma": 
            status = "pass" 
        else: 
            status = "fail"

        evidence_str =  "Faltantes: " +  str_missing_columns + " Nomes_duplicados: " + resultado_string          
        RESULTS.append(save_result(row["file"], "Todos" ,"structure","Column info",evidence_str,"", status)) 

        # Chama as rotinas de nivel de arquivo 
        for index, row in df_file_checks.iterrows():
            getattr(src.analisys,df_file_checks.loc[0, "routine"])(df_data, df_fields, row)
            
        new_file = False

print("\n--- REGISTROS DE AUDITORIA ---")        

if RESULTS: # Verifica se a lista não está vazia
    DF_RESULTS = pd.DataFrame(RESULTS)
    # Prossiga com o seu código...
    print(DF_RESULTS[['status', 'file', 'Field', 'test', 'detail', 'evidence']])
else:
    print("A lista 'result' está vazia. Nenhum registro de auditoria foi criado.")



CARREGOU!
------------------------------
ses_seguros
ses_seguros.csv
------------------------------
CARREGOU!
------------------------------
ses_cias
ses_cias.csv
------------------------------
CARREGOU!
------------------------------
ses_ramos
ses_ramos.csv
------------------------------

--- REGISTROS DE AUDITORIA ---
  status             file  Field         test detail  \
0   pass  ses_seguros.csv  Todos    file info          
1   pass  ses_seguros.csv  Todos  Column info          
2   pass     ses_cias.csv  Todos    file info          
3   fail     ses_cias.csv  Todos  Column info          
4   pass    ses_ramos.csv  Todos    file info          
5   fail    ses_ramos.csv  Todos  Column info          

                                      evidence  
0      Tamanho: 7.56 KB Linhas/Colunas: 100/21  
1  Faltantes: nenhuma Nomes duplicados: nenhum  
2      Tamanho: 59.51 KB Linhas/Colunas: 743/5  
3  Faltantes: nenhuma Nomes duplicados: noenti  
4       Tamanho: 6.96 KB Linhas/Colunas: